## Definição das variaveis globais utilizadas

In [1]:
dataName    = "PETR4_B_0_30min"            # Name of the data file
setDivision = [0.1, 0.7, 0.2]              # Size of the [optimization, train, test] set
outputName  = "Fechamento"                 # Name of the output variable0

#### Bibliotecas que devem ser instaladas para a execução do trabalho

In [2]:
%pip install pandas
%pip install scikit-learn
%pip install scipy
%pip install enum
%pip install enum34
%pip install numpy
%pip install matplotlib
%pip install statsmodels
%pip install arch
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached enum-0.4.7.tar.gz (20 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [27 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 11, in <module>
        File "/usr/lib/python3.10/tokenize.py", line 33, in <module>
          import re
        File "/usr/lib/python3.

#### Versões de pacotes utilizadas

In [3]:
import pandas as pd
import numpy  as np
import sklearn as sk
import matplotlib as mt
import sys

print("Python:  ", sys.version)
print("Pandas:  ", pd.__version__)
print("Numpy:   ", np.__version__)
print("Matplt:  ", mt.__version__)
print("Sklearn: ", sk.__version__)

Python:   3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
Pandas:   2.0.3
Numpy:    1.24.3
Matplt:   3.5.1
Sklearn:  1.3.0


## Criando a base de dados

#### Métodos  disponíveis
- SMA => Média móvel simples = $\frac{1}{n} \sum_{i=0}^{n-1} Price_{t-x}$ 
    - x => tamanho da janela de amostragem
- EMA => Média móvel exponencial = $\alpha (currentPrice - EMA_{t-1}) + EMA_{t-1}$
    - $\alpha = \frac{2}{n+1}$ => Fator de suavização
    - n => número de amostras na análise
    - $EMA_{t-1}$ => média móvel exponencial anterior
- MACD => Convergência/Divergência da média móvel = $EMA(n) - EMA(k)$
    - n => média exponencial rápida
    - k => média exponencial lenta
- CCI => Índice de Canal de Commodities = $\frac{TP - SMA(TP,N)}{0.015 * DP(TP)}$
    - DP => Desvio padrão de TP
    - SMA => Média móvel simples
    - TP => Preço típico (uma média simples entre os valores de fechamento, alta e baixa)
- ADX => Indicador de força da tendência = $\frac{(n-1)*EMA(TR) + TR}{n}$
    - TR => True Range = $max(H-L, abs(H-C_{t-1}), abs(L-C_{t-1}))$
- MTM => Indicador de momento = $Price_{t}-Price_{t-n}$
- ROC => Taxa de variação = $\frac{Price_{t}-Price_{t-n}}{Price_{t-n}}100$
- TSI => Indicador de Força Real = $\frac{EMA(EMA(PC,n), k)}{EMA(EMA(|PC|,n), k)}100$
    - PC => Variação do preço de fechamento = $price_{t-1} - price_t$
- K (%K) => Oscilador estocástico = $\frac{C_t-L_{t-n}}{H_{t-n}-L_{t-n}}$
    - C => Preço de fechamento
    - L => Menor valor
    - H => Maior valor
- D (%D) => Média móvel simples do Oscilador estocástico = $\frac{\sum_{i=0}^{n-1} \%K_{t-i}}{n}$
    - %K => Oscilador estocástico
- R (%R) => Williams %R = $\frac{H_{t-n}-C_t}{H_{t-n}-L_{t-n}}$ 
    - C => Preço de fechamento
    - L => Menor valor
    - H => Maior valor


#### Gerando base de dados

In [4]:
from data_generate import Generate

dataShape, outShape = Generate(dataName, outputName)
print("Data shape: ", dataShape)
print("Output shape: ", outShape)

AttributeError: 'DataFrame' object has no attribute 'map'

#### Selecionando as variáveis mais relevantes

* Randon Forest => Gera vários inputs de dados a partir de um método randomico e cria uma árvore binaria para cada um deles, realizando assim a predição com os melhores resultados (É possível treinar esse modelo e verificar quais foram as variaveis mais importantes para ele). 
* Fisher => Calcula o escore de Fisher para selecionar as caracteristicas mais importantes de um conjunto de dados, basicamente ele realiza a divizão do "erro médio quadratico ponderado" pela "variância ponderada" (Essa implementação foca em algoritmos de classificação com classes definidas por inteiros).
* Gini => calcula o índice de impureza de Gini para cada feature (variável independente) em relação à variável dependente Y. O índice de impureza de Gini é uma medida de quão impura é a divisão das amostras em duas classes, e é usado em árvores de decisão para selecionar o melhor atributo para dividir os dados em subconjuntos mais homogêneos (Utilizado em bases de dados definidas para classificação com classes referênciadas em int).
* KruskalWallis => É um teste não-paramétrico que compara as medianas de duas ou mais amostras independentes para determinar se há diferenças estatisticamente significativas entre elas (Pega os valores em que a maior próximidade estatistica -> não implica diretamente que possa ser uma boa escolha para o modelo).
* Lasso => (Least Absolute Shrinkage and Selection Operator) é um método de regressão linear que utiliza a regularização L1 para selecionar as variáveis mais importantes em um conjunto de dados. O objetivo do Lasso é minimizar a soma dos erros quadrados (Caráter linear).
* ElasticNet => é uma extensão do algoritmo Lasso (Least Absolute Shrinkage and Selection Operator) e Ridge Regression, que combina a penalização L1 e L2 para selecionar variáveis importantes em um modelo de regressão linear. É utilizado para lidar com problemas de regressão em que há um grande número de variáveis preditoras (high-dimensional data) e muitas dessas variáveis podem não ser relevantes para a predição da variável resposta (Caráter linear).
* Chi2 ($X^2$) =>  é um teste estatístico que é usado para determinar se existe uma relação significativa entre duas variáveis categóricas. Ele é usado para avaliar se as diferenças entre as frequências observadas e as frequências esperadas são significativas o suficiente para rejeitar a hipótese nula de que não há relação entre as duas variáveis.
* F_Regression => Ele utiliza a estatística F de Fisher para avaliar a relação linear entre as variáveis e calcular o p-valor associado a cada feature.
* Mutual_Info_Regression => É uma técnica de seleção de recursos que mede a dependência mútua entre cada recurso e a variável de destino, usando a entropia da informação. Ele estima a informação mútua entre cada recurso e o destino, para ajudar na seleção de recursos que são relevantes para a predição do alvo.

In [ ]:
from data_selection import Selection

d1Shape, d2Shape, outShape = Selection(dataName)
print("Dataset 1 shape: ", d1Shape)
print("Dataset 2 shape: ", d2Shape)
print("Output shape: ", outShape)

#### Separando a base de dados
* Otimização: base de dados destinada ao processo de otimização dos hiperparâmetros das redes.
* Treinamento: base de dados destinada ao processo de treinamento dos modelos de IA.
* Teste: base de dados destinada a realizar a avalição de mensurar o desempenho da proposta

In [ ]:
from data_cut import Cut
Cut(dataName, setDivision)

## Modelos de IA utilizados
* Conjunto de classificação:
    - Support Vector Machine (SVM)
    - K Neighbors Classifier (KNN)
    - Logistic Refression (LR)
* Conjunto Estatístico:
    - Generalized Autoregressive Conditional Heteroskedasticity (GARCH)
    - Autoregressive Integrated Moving Average (ARIMA)
    - Seasonal Autoregressive Integreted Moving Average (SARIMA)
* Conjunto de Regressão:
    - Long Short-Term Memory (LSTM)
    - Convolutional Neural Network (CNN)
    - Recurrent Neural Network (RNN)

#### Obtendo modelos otimizados
##### Modelos de Classificação

In [ ]:
from models_classification import GetModelsClassificationOptimized
SVM, KNN, LR = GetModelsClassificationOptimized(dataName, setDivision[0])

##### Modelos Estatísticos

In [ ]:
from models_statistic import GetModelsStatisticsOptimized
import warnings
warnings.filterwarnings("ignore")

ARIMA, SARIMA, GARCH = GetModelsStatisticsOptimized(dataName, setDivision[0])


##### Modelos de Regressão

In [ ]:
from models_regression import GetModelsRegressionOptimized
LSTM, CNN, RNN = GetModelsRegressionOptimized(dataName, setDivision[0])

### Recuperando Modelos já Otimizados

In [ ]:
from models_classification import GetModelsClassification
from models_statistic import GetModelsStatistics
from models_regression import GetModelsRegression

SVM, KNN, LR = GetModelsClassification(dataName)
ARIMA, SARIMA, GARCH = GetModelsStatistics(dataName)
LSTM, CNN, RNN = GetModelsRegression(dataName)

In [ ]:
from models_statistic import GetStatisticPredictions
from models_classification import GetClassificationPredictions
from models_regression import GetRegressionPredictions
import pandas as pd

Y_Train_statistic = pd.read_csv(f'../Data/Cut/statistic/Y/Train_{setDivision[1]}{dataName}.csv', sep=";")['OutPut |T+1|']
Y_Test_statistic  = pd.read_csv(f'../Data/Cut/statistic/Y/Test_{setDivision[2]}{dataName}.csv', sep=";")['OutPut |T+1|']

X_Train_dataset1 = pd.read_csv(f'../Data/Cut/dataset1/X/Train_{setDivision[1]}{dataName}.csv', sep=";")
Y_Train_dataset1 = pd.read_csv(f'../Data/Cut/dataset1/Y/Train_{setDivision[1]}{dataName}.csv', sep=";")['OutPut_class |T+1|']
X_Test_dataset1  = pd.read_csv(f'../Data/Cut/dataset1/X/Test_{setDivision[2]}{dataName}.csv', sep=";")
Y_Test_dataset1  = pd.read_csv(f'../Data/Cut/dataset1/Y/Test_{setDivision[2]}{dataName}.csv', sep=";")['OutPut_class |T+1|']

X_Train_dataset2 = pd.read_csv(f'../Data/Cut/dataset2/X/Train_{setDivision[1]}{dataName}.csv', sep=";")
Y_Train_dataset2 = pd.read_csv(f'../Data/Cut/dataset2/Y/Train_{setDivision[1]}{dataName}.csv', sep=";")['OutPut |T+1|']
X_Test_dataset2  = pd.read_csv(f'../Data/Cut/dataset2/X/Test_{setDivision[2]}{dataName}.csv', sep=";")
Y_Test_dataset2  = pd.read_csv(f'../Data/Cut/dataset2/Y/Test_{setDivision[2]}{dataName}.csv', sep=";")['OutPut |T+1|']

LSTM_epoch = pd.read_csv(f'../Results/optimization/regression/LSTM/{dataName}_Logs.csv', sep=";")['Epochs'][0]
LSTM_batch = pd.read_csv(f'../Results/optimization/regression/LSTM/{dataName}_Logs.csv', sep=";")['Batch Size'][0]
CNN_epoch  = pd.read_csv(f'../Results/optimization/regression/CNN/{dataName}_Logs.csv', sep=";")['Epochs'][0]
CNN_batch  = pd.read_csv(f'../Results/optimization/regression/CNN/{dataName}_Logs.csv', sep=";")['Batch Size'][0]
RNN_epoch  = pd.read_csv(f'../Results/optimization/regression/RNN/{dataName}_Logs.csv', sep=";")['Epochs'][0]
RNN_batch  = pd.read_csv(f'../Results/optimization/regression/RNN/{dataName}_Logs.csv', sep=";")['Batch Size'][0]

ClassificationModels = [SVM, KNN, LR]
RegressionModels = [LSTM, CNN, RNN]
RegressionNames  = ['LSTM', 'CNN', 'RNN']
RegressionEpochs = [LSTM_epoch, CNN_epoch, RNN_epoch]
RegressionBatch  = [LSTM_batch, CNN_batch, RNN_batch]

GetStatisticPredictions(dataName, Y_Train_statistic.ravel(), Y_Test_statistic.ravel(), window=400)
GetClassificationPredictions(dataName, ClassificationModels, X_Test_dataset1, Y_Test_dataset1, X_Train_dataset1, Y_Train_dataset1)
GetRegressionPredictions(dataName, RegressionNames, RegressionModels, RegressionEpochs, RegressionBatch, X_Test_dataset2, Y_Test_dataset2, X_Train_dataset2, Y_Train_dataset2)


In [ ]:
from strategy import GetEnsambles
GetEnsambles(dataName, setDivision[2])

In [ ]:
from analyze import MakeClassificationsLogs
MakeClassificationsLogs(dataName, setDivision[2])